In [1]:
using WAV, Knet

In [2]:
music_sample_size = 479232
sample_frequency = 16000
batch_size = 12

function load_musician_dataset()
    # Some constants we need to define now
    music_sample_size = 479232 #How long our "songs" are

    # Directories holding each musician's subdirectory of samples
    use_musicians = ["satriani"]
    musician_dirs = filter(x -> x in use_musicians, readdir("/mnt/data/other/music_samples/processed/"))

    # Arrays which will hold the data we read from the wav files
    X_train = Array{Array{Float32,1},1}()
    Y_train = Array{UInt16,1}()
    X_test = Array{Array{Float32,1},1}()
    Y_test = Array{UInt16,1}()

    # Mapping for the musician labels
    musician_mapping = Dict{String,UInt16}()
    musician_label = 0

    for musician in musician_dirs

        # Add musician to the label mapping, with an incrementing index
        musician_label += 1
        musician_mapping[musician] = musician_label
       
        # Read the samples for each musician into X and associate the apropriate label into Y
        index = 0
        for sample in readdir("/mnt/data/other/music_samples/processed/$(musician)")
            values_raw, fq = wavread("/mnt/data/other/music_samples/processed/$(musician)/$(sample)")
            values_mon = values_raw[:,1]
            if  round(rand()*7 + 1) == 3
                if length(values_mon) >= music_sample_size
                    push!(X_test, values_mon[1:music_sample_size])
                    push!(Y_test, musician_label)
                end
            else
                if length(values_mon) >= music_sample_size
                    push!(X_train, values_mon[1:music_sample_size])
                    push!(Y_train, musician_label)
                end
            end
        end
    end
    
    X_test_matrix = Array{Float32,2}(hcat(X_test...))
    X_train_matrix = Array{Float32,2}(hcat(X_train...))
    
    return minibatch(X_train_matrix, Y_train, batch_size, shuffle=true), minibatch(X_test_matrix, Y_test, batch_size, shuffle=true)
end

load_musician_dataset (generic function with 1 method)

In [3]:
Dtrain, Dtest = load_musician_dataset()

(Knet.MB(Float32[0.0 0.1377 … -0.0993988 -0.104984; 0.0 0.104648 … -0.0798669 -0.115879; … ; 0.196844 -0.119938 … -0.169195 0.000183111; 0.121464 -0.120945 … -0.170507 -0.000122074], UInt16[0x0001 0x0001 … 0x0001 0x0001], 12, 83, false, [60, 82, 57, 29, 75, 34, 6, 41, 18, 47  …  26, 7, 64, 67, 20, 73, 36, 50, 31, 38], [479232, 83], [83], Array{Float32,2}, Array{UInt16,1}), Knet.MB(Float32[0.106998 -0.098178 … -0.100131 -0.0415967; 0.255898 -0.0116581 … -0.0743431 -0.131138; … ; -0.355144 -0.0723899 … -0.437269 -0.148717; -0.471267 0.0218818 … -0.102206 -0.149327], UInt16[0x0001 0x0001 … 0x0001 0x0001], 12, 12, false, [6, 5, 4, 9, 2, 7, 3, 8, 12, 10, 1, 11], [479232, 12], [12], Array{Float32,2}, Array{UInt16,1}))

In [4]:
function create_discriminator()
    samples = music_sample_size
    Weights = [
        xavier(Float32,Int64(samples*2/(3*512)),samples),
        xavier(Float32,Int64(samples/1024),Int64(samples*2/(3*512))), 
        xavier(Float32,Int64(samples/2048),Int64(samples/1024)), 
        xavier(Float32,2,Int64(samples/2048))
    ]
    Biases = [
        zeros(Float32,samples*2/(3*512),1),
        zeros(Float32,samples/1024,1),
        zeros(Float32,samples/2048,1),
        zeros(Float32,2,1)
    ]
    Model = Any[]
    for i = 1:length(Weights)
        push!(Model, Weights[i])
        push!(Model, Biases[i])
    end
    Model
end

function create_generator()
    granularity = 64
    oscilators = 80
    samples = music_sample_size
    Weights = [
        xavier(Float32,Int64(oscilators * granularity), Int64(oscilators * granularity)),
        xavier(Float32,Int64(oscilators * granularity / 2), Int64(oscilators * granularity)), 
        xavier(Float32,Int64(oscilators * granularity / 4), Int64(oscilators * granularity / 2)), 
        xavier(Float32,Int64(oscilators * granularity), Int64(oscilators * granularity / 4))
    ]
    Biases = [
        zeros(Float32,Int64(oscilators * granularity), 1),
        zeros(Float32,Int64(oscilators * granularity / 2), 1),
        zeros(Float32,Int64(oscilators * granularity / 4), 1),
        zeros(Float32,Int64(oscilators * granularity), 1)
    ]
    Model = Any[]
    for i = 1:length(Weights)
        push!(Model, Weights[i])
        push!(Model, Biases[i])
    end
    Model
end

create_generator (generic function with 1 method)

In [40]:
function discriminate(Model,X)
    for i=1:2:length(Model) - 2
        X = tanh.(Model[i]*X .+ Model[i+1])
    end
    labels = tanh.(Model[end - 1]*X .+ Model[end])
    return labels
end
#=
function generate(Model, batch_size)
    Z = Array(randn(Float32, music_sample_size, batch_size))
    for i=1:2:length(Model) - 2
        Z = tanh.(Model[i]*Z .+ Model[i+1])
    end
    labels = tanh.(Model[end - 1]*Z .+ Model[end])
    return labels
end
=#
granularity = 64
oscilators = 80
function generate(Model, bs)
    Z = Array(zeros(Float32, granularity*oscilators, 1)) + 0.1
    
    for i=1:2:length(Model) - 2
        Z = tanh.(Model[i]*Z .+ Model[i+1])
    end
    Z = tanh.(Model[end - 1]*Z .+ Model[end])
    
    samples = Array(zeros(Float32, music_sample_size, 1))
    
    vals = [] 
    for sample_nr=0:oscilators:oscilators*(granularity - 1)
        push!(vals, Z[(sample_nr + 1):(sample_nr + oscilators)])        
    end
    
    for si=1:music_sample_size
        for v in vals[Int64(round(1 + si/music_sample_size * (granularity -1) ))]
            samples[si] += sin(si*v/sample_frequency)
        end
        samples[si] = samples[si]/oscilators
    end
    return hcat(samples,samples,samples,samples)
end


generate (generic function with 2 methods)

In [ ]:
D = create_discriminator()
G = create_generator()

function loss_d(D,X,Y_real) 
    return nll(discriminate(D,X), Y_real)
end 

lossgradients_d = grad(loss_d)

function loss_g(D,FS,TY)
    return nll(discriminate(D,FS), TY)
end

lossgradients_g = grad(loss_g)

optimizers_d = optimizers(D, Nesterov)
optimizers_g = optimizers(G, Nesterov)


function save_samples(epoch)
    samples = generate(G, 5)
    for i=1:length(samples[1,:])
        wavwrite(samples[:,i], 16000, "/mnt/data/other/music_samples/generated/epoch-$(epoch)-nr-$(i).wav")
        println("Generated a sample !")
    end
end

for epoch=1:20
    println("Started training for epoch: $(epoch)")
    
    for (X,Y) in Dtrain
        
        fake_samples = generate(G, batch_size)
        fake_Y = zeros(UInt16, length(fake_samples[1,:]))
        
        X = hcat(X, fake_samples)
        Y = vcat(Y, fake_Y)
        
        GD = lossgradients_d(D,X,Y)
        
        fake_samples = generate(G, 4)
        wannabe_Y = [UInt16(1),UInt16(1),UInt16(1),UInt16(1)]
        GG = lossgradients_g(D,fake_samples, wannabe_Y)
        println(GG)
        update!(D, GD , optimizers_d)
        update!(G, GG , optimizers_g)
        save_samples(epoch)
    end
    
    println("Finished training epoch: $(epoch)")
    println("Got accuracy: $(accuracy(D, Dtrain, discriminate)) , on the training set")
    save_samples(epoch)
    flush(STDOUT)
end

Started training for epoch: 1


Any[Float32[7.73737f-10 1.54747f-9 2.32121f-9 3.09495f-9 3.86869f-9 4.64242f-9 5.41616f-9 6.1899f-9 6.96364f-9 7.73737f-9 8.51111f-9 9.28485f-9 1.00586f-8 1.08323f-8 1.16061f-8 1.23798f-8 1.31535f-8 1.39273f-8 1.4701f-8 1.54747f-8 1.62485f-8 1.70222f-8 1.7796f-8 1.85697f-8 1.93434f-8 2.01172f-8 2.08909f-8 2.16646f-8 2.24384f-8 2.32121f-8 2.39859f-8 2.47596f-8 2.55333f-8 2.63071f-8 2.70808f-8 2.78545f-8 2.86283f-8 2.9402f-8 3.01757f-8 3.09495f-8 3.17232f-8 3.2497f-8 3.32707f-8 3.40445f-8 3.48182f-8 3.55919f-8 3.63656f-8 3.71394f-8 3.79131f-8 3.86869f-8 3.94606f-8 4.02344f-8 4.10081f-8 4.17818f-8 4.25556f-8 4.33293f-8 4.4103f-8 4.48768f-8 4.56505f-8 4.64242f-8 4.7198f-8 4.79717f-8 4.87454f-8 4.95192f-8 5.02929f-8 5.10667f-8 5.18404f-8 5.26141f-8 5.33879f-8 5.41616f-8 5.49353f-8 5.57091f-8 5.64828f-8 5.72565f-8 5.80303f-8 5.8804f-8 5.95778f-8 6.03515f-8 6.11253f-8 6.1899f-8 6.26727f-8 6.34464f-8 6.42202f-8 6.49939f-8 6.57677f-8 6.65414f-8 6.73151f-8 6.80889f-8 6.88626f-8 6.96364f-8 7.0410

1.45308f-6 1.45385f-6 1.45462f-6 1.4554f-6 1.45617f-6 1.45695f-6 1.45772f-6 1.45849f-6 1.45927f-6 1.46004f-6 1.46081f-6 1.46159f-6 1.46236f-6 1.46314f-6 1.46391f-6 1.46468f-6 1.46546f-6 1.46623f-6 1.467f-6 1.46778f-6 1.46855f-6 1.46933f-6 1.4701f-6 1.47087f-6 1.47165f-6 1.47242f-6 1.47319f-6 1.47397f-6 1.47474f-6 1.47552f-6 1.47629f-6 1.47706f-6 1.47784f-6 1.47861f-6 1.47938f-6 1.48016f-6 1.48093f-6 1.4817f-6 1.48248f-6 1.48325f-6 1.48403f-6 1.4848f-6 1.48557f-6 1.48635f-6 1.48712f-6 1.48789f-6 1.48867f-6 1.48944f-6 1.49022f-6 1.49099f-6 1.49176f-6 1.49254f-6 1.49331f-6 1.49409f-6 1.49486f-6 1.49563f-6 1.49641f-6 1.49718f-6 1.49795f-6 1.49873f-6 1.4995f-6 1.50027f-6 1.50105f-6 1.50182f-6 1.5026f-6 1.50337f-6 1.50414f-6 1.50492f-6 1.50569f-6 1.50646f-6 1.50724f-6 1.50801f-6 1.50879f-6 1.50956f-6 1.51033f-6 1.51111f-6 1.51188f-6 1.51265f-6 1.51343f-6 1.5142f-6 1.51498f-6 1.51575f-6 1.51652f-6 1.5173f-6 1.51807f-6 1.51884f-6 1.51962f-6 1.52039f-6 1.52117f-6 1.52194f-6 1.52271f-6 1.52349f-

-6 2.18116f-6 2.18193f-6 2.18271f-6 2.18348f-6 2.18426f-6 2.18503f-6 2.1858f-6 2.18658f-6 2.18735f-6 2.18812f-6 2.1889f-6 2.18967f-6 2.19044f-6 2.19122f-6 2.19199f-6 2.19277f-6 2.19354f-6 2.19431f-6 2.19509f-6 2.19586f-6 2.19664f-6 2.19741f-6 2.19818f-6 2.19896f-6 2.19973f-6 2.2005f-6 2.20128f-6 2.20205f-6 2.20282f-6 2.2036f-6 2.20437f-6 2.20514f-6 2.20592f-6 2.20669f-6 2.20747f-6 2.20824f-6 2.20901f-6 2.20979f-6 2.21056f-6 2.21133f-6 2.21211f-6 2.21288f-6 2.21366f-6 2.21443f-6 2.2152f-6 2.21598f-6 2.21675f-6 2.21752f-6 2.2183f-6 2.21907f-6 2.21985f-6 2.22062f-6 2.22139f-6 2.22217f-6 2.22294f-6 2.22372f-6 2.22449f-6 2.22526f-6 2.22604f-6 2.22681f-6 2.22758f-6 2.22836f-6 2.22913f-6 2.22991f-6 2.23068f-6 2.23145f-6 2.23223f-6 2.233f-6 2.23377f-6 2.23455f-6 2.23532f-6 2.23609f-6 2.23687f-6 2.23764f-6 2.23842f-6 2.23919f-6 2.23996f-6 2.24074f-6 2.24151f-6 2.24228f-6 2.24306f-6 2.24383f-6 2.24461f-6 2.24538f-6 2.24615f-6 2.24693f-6 2.2477f-6 2.24848f-6 2.24925f-6 2.25002f-6 2.2508f-6 2.2515

2.90846f-6 2.90924f-6 2.91001f-6 2.91079f-6 2.91156f-6 2.91233f-6 2.91311f-6 2.91388f-6 2.91466f-6 2.91543f-6 2.9162f-6 2.91698f-6 2.91775f-6 2.91852f-6 2.9193f-6 2.92007f-6 2.92085f-6 2.92162f-6 2.92239f-6 2.92316f-6 2.92394f-6 2.92471f-6 2.92549f-6 2.92626f-6 2.92703f-6 2.92781f-6 2.92858f-6 2.92936f-6 2.93013f-6 2.9309f-6 2.93168f-6 2.93245f-6 2.93323f-6 2.934f-6 2.93477f-6 2.93554f-6 2.93632f-6 2.93709f-6 2.93787f-6 2.93864f-6 2.93941f-6 2.94019f-6 2.94096f-6 2.94173f-6 2.94251f-6 7.24618f-7 7.24808f-7 7.24998f-7 7.25188f-7 7.2538f-7 7.25571f-7 7.2576f-7 7.2595f-7 7.26141f-7 7.26331f-7 7.26522f-7 7.26712f-7 7.26903f-7 7.27094f-7 7.27284f-7 7.27475f-7 7.27666f-7 7.27855f-7 7.28046f-7 7.28236f-7 7.28428f-7 7.28618f-7 7.28808f-7 7.28999f-7 7.29189f-7 7.29379f-7 7.2957f-7 7.29761f-7 7.29951f-7 7.30141f-7 7.30332f-7 7.30523f-7 7.30713f-7 7.30903f-7 7.31093f-7 7.31284f-7 7.31475f-7 7.31666f-7 7.31856f-7 7.32046f-7 7.32236f-7 7.32428f-7 7.32618f-7 7.32809f-7 7.32999f-7 7.3319f-7 7.33381f-

...

.28338f-5 -1.28341f-5 -1.28344f-5 -1.28347f-5 -1.2835f-5 -1.28353f-5 -1.28356f-5 -1.28359f-5 -1.28362f-5 -1.28365f-5 -1.28368f-5 -1.28371f-5 -1.28374f-5 -1.28377f-5 -1.2838f-5 -1.28383f-5 -1.28386f-5 -1.28389f-5 -1.28392f-5 -1.28395f-5 -1.28398f-5 -1.28401f-5 -1.28404f-5 -1.28407f-5 -1.2841f-5 -1.28413f-5 -1.28416f-5 -1.28419f-5 -1.28422f-5 -1.28425f-5 -1.28428f-5 -1.28431f-5 -1.28434f-5 -1.28437f-5 -1.2844f-5 -1.28443f-5 -1.28446f-5 -1.28449f-5 -1.28452f-5 -1.28455f-5 -1.28458f-5 -1.28461f-5 -1.28464f-5 -1.28467f-5 -1.2847f-5 -1.28473f-5 -1.28476f-5 -1.28479f-5 -1.28482f-5 -1.28485f-5 -1.28488f-5 -1.28491f-5 -1.28494f-5 -1.28497f-5 -1.285f-5 -1.28503f-5 -1.28506f-5 -1.28509f-5 -1.28512f-5 -1.28515f-5 -1.28518f-5 -1.28521f-5 -1.28524f-5 -1.28527f-5 -1.2853f-5 -1.28533f-5 -1.28536f-5 -1.28539f-5 -1.28542f-5 -1.28545f-5 -1.28548f-5 -1.28551f-5 -1.28554f-5 -1.28557f-5 -1.2856f-5 -1.28563f-5 -1.28566f-5 -1.28569f-5 -1.28572f-5 -1.28575f-5 -1.28578f-5 -1.28581f-5 -1.28584f-5 -1.28587f-5 -1.

30927f-5 -1.3093f-5 -1.30933f-5 -1.30936f-5 -1.30939f-5 -1.30942f-5 -1.30945f-5 -1.30948f-5 -1.30951f-5 -1.30954f-5 -1.30957f-5 -1.3096f-5 -1.30963f-5 -1.30966f-5 -1.30969f-5 -1.30972f-5 -1.30975f-5 -1.30978f-5 -1.30981f-5 -1.30984f-5 -1.30987f-5 -1.3099f-5 -1.30993f-5 -1.30996f-5 -1.30999f-5 -1.31002f-5 -1.31005f-5 -1.31008f-5 -1.31011f-5 -1.31014f-5 -1.31017f-5 -1.3102f-5 -1.31023f-5 -1.31026f-5 -1.31029f-5 -1.31032f-5 -1.31035f-5 -1.31038f-5 -1.31041f-5 -1.31044f-5 -1.31047f-5 -1.3105f-5 -1.31053f-5 -1.31056f-5 -1.31059f-5 -1.31062f-5 -1.31065f-5 -1.31068f-5 -1.31071f-5 -1.31074f-5 -1.31077f-5 -1.3108f-5 -1.31083f-5 -1.31086f-5 -1.31089f-5 -1.31092f-5 -1.31095f-5 -1.31098f-5 -1.31101f-5 -1.31104f-5 -1.31107f-5 -1.3111f-5 -1.31113f-5 -1.31116f-5 -1.3112f-5 -1.31123f-5 -1.31126f-5 -1.31129f-5 -1.31132f-5 -1.31135f-5 -1.31138f-5 -1.31141f-5 -1.31144f-5 -1.31147f-5 -1.3115f-5 -1.31153f-5 -1.31156f-5 -1.31159f-5 -1.31162f-5 -1.31165f-5 -1.31168f-5 -1.31171f-5 -1.31174f-5 -1.31177f-5 -1.3

f-5 -1.3352f-5 -1.33523f-5 -1.33526f-5 -1.33529f-5 -1.33532f-5 -1.33535f-5 -1.33538f-5 -1.33541f-5 -1.33544f-5 -1.33547f-5 -1.3355f-5 -1.33553f-5 -1.33556f-5 -1.33559f-5 -1.33562f-5 -1.33565f-5 -1.33568f-5 -1.33571f-5 -1.33574f-5 -1.33577f-5 -1.3358f-5 -1.33583f-5 -1.33586f-5 -1.33589f-5 -1.33592f-5 -1.33595f-5 -1.33598f-5 -1.33601f-5 -1.33604f-5 -1.33607f-5 -1.3361f-5 -1.33613f-5 -1.33616f-5 -1.33619f-5 -1.33622f-5 -1.33625f-5 -1.33628f-5 -1.33631f-5 -1.33634f-5 -1.33637f-5 -1.3364f-5 -1.33643f-5 -1.33646f-5 -1.33649f-5 -1.33652f-5 -1.33655f-5 -1.33658f-5 -1.33661f-5 -1.33664f-5 -1.33667f-5 -1.3367f-5 -1.33673f-5 -1.33676f-5 -1.33679f-5 -1.33682f-5 -1.33685f-5 -1.33688f-5 -1.33691f-5 -1.33694f-5 -1.33697f-5 -1.337f-5 -1.33703f-5 -1.33706f-5 -1.33709f-5 -1.33712f-5 -1.33715f-5 -1.33718f-5 -1.33721f-5 -1.33724f-5 -1.33727f-5 -1.3373f-5 -1.33733f-5 -1.33736f-5 -1.33739f-5 -1.33742f-5 -1.33745f-5 -1.33748f-5 -1.33751f-5 -1.33754f-5 -1.33757f-5 -1.3376f-5 -1.33763f-5 -1.33766f-5 -1.33769f-

-1.36109f-5 -1.36112f-5 -1.36115f-5 -1.36118f-5 -1.36121f-5 -1.36124f-5 -1.36127f-5 -1.3613f-5 -1.36133f-5 -1.36136f-5 -1.36139f-5 -1.36142f-5 -1.36145f-5 -1.36148f-5 -1.36151f-5 -1.36155f-5 -1.36157f-5 -1.36161f-5 -1.36164f-5 -1.36167f-5 -1.3617f-5 -1.36173f-5 -1.36176f-5 -1.36179f-5 -1.36182f-5 -1.36185f-5 -1.36188f-5 -1.36191f-5 -1.36194f-5 -1.36197f-5 -1.362f-5 -1.36203f-5 -1.36206f-5 -1.36209f-5 -1.36212f-5 -1.36215f-5 -1.36218f-5 -1.36221f-5 -1.36224f-5 -1.36227f-5 -1.3623f-5 -1.36233f-5 -1.36236f-5 -1.36239f-5 -1.36242f-5 -1.36245f-5 -1.36248f-5 -1.36251f-5 -1.36254f-5 -1.36257f-5 -1.3626f-5 -1.36263f-5 -1.36266f-5 -1.36269f-5 -1.36272f-5 -1.36275f-5 -1.36278f-5 -1.36281f-5 -1.36284f-5 -1.36287f-5 -1.3629f-5 -1.36293f-5 -1.36296f-5 -1.36299f-5 -1.36302f-5 -1.36305f-5 -1.36308f-5 -1.36311f-5 -1.36314f-5 -1.36317f-5 -1.3632f-5 -1.36323f-5 -1.36326f-5 -1.36329f-5 -1.36332f-5 -1.36335f-5 -1.36338f-5 -1.36341f-5 -1.36344f-5 -1.36347f-5 -1.3635f-5 -1.36353f-5 -1.36356f-5 -1.36359f-5 -

Excessive output truncated after 524289 bytes.